In [98]:
import sys
sys.path.append('../')


In [3]:
import spacy
import numpy as np
np.random.seed(0)
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform

from nlp_helper import read_glove_vecs, convert_to_one_hot

Using TensorFlow backend.


In [4]:
nlp=spacy.load('en')

In [5]:
#加载训练数据
pos_datas = np.load("../../data/twitter/tech_not/pos_cleaned.npy")
neg_datas = np.load("../../data/twitter/tech_not/neg_cleaned.npy")

np.random.seed(1)
np.random.shuffle(pos_datas)
np.random.shuffle(neg_datas)

pos_datas_train, pos_datas_test = pos_datas[:700],pos_datas[700:800]
neg_datas_train_10, neg_datas_test_10 = neg_datas[:7000],neg_datas[7000:8000]
neg_datas_train_5, neg_datas_test_5 = neg_datas[13600:17100],neg_datas[17100:20000]
neg_datas_train_1_2 = neg_datas[20000:20350]
neg_datas_train_2, neg_datas_test_2 = neg_datas[12000:13400],neg_datas[13400:13600]

In [22]:
w_to_ix, _, w_to_vec_map  = read_glove_vecs("../../data/embedding/glove.twitter.27B.50d.txt")

In [9]:
# train1_2 = np.concatenate((pos_datas_train,neg_datas_train_2), axis = 0)

# np.random.shuffle(train1_2)
# X_train_1_2 = train1_2[:,1]
# Y_train_1_2 = np.array(list(map(int, train1_2[:,0])))
# train1_5 = np.concatenate((pos_datas_train,neg_datas_train_5), axis = 0)

# np.random.shuffle(train1_5)
# X_train_1_5 = train1_5[:,1]
# Y_train_1_5 = np.array(list(map(int, train1_5[:,0])))

In [10]:
pos_datas_50augs = np.tile(pos_datas, (50,1))


In [11]:
train_auged = np.concatenate((pos_datas_50augs,neg_datas), axis = 0)

np.random.shuffle(train_auged )

X_train, X_test = train_auged[:,1][0:80000], train_auged[:,1][80000:]
Y_train, Y_test = np.array(list(map(int, train_auged[:,0])))[0:80000], np.array(list(map(int, train_auged[:,0])))[80000:]

In [102]:
test_data = np.concatenate((pos_datas, neg_datas), axis = 0)
X_test_origin, Y_test_origin = test_data[:,1], np.array(list(map(int, test_data[:,0])))

In [64]:
def sentences_to_indices(X, word_to_index, max_len):
    """
    Converts an array of sentences (strings) into an array of indices corresponding to words in the sentences.
    The output shape should be such that it can be given to `Embedding()` (described in Figure 4). 
    
    Arguments:
    X -- array of sentences (strings), of shape (m, 1)
    word_to_index -- a dictionary containing the each word mapped to its index
    max_len -- maximum number of words in a sentence. You can assume every sentence in X is no longer than this. 
    
    Returns:
    X_indices -- array of indices corresponding to words in the sentences from X, of shape (m, max_len)
    """
    
    m = X.shape[0]                                   # number of training examples
    
    # Initialize X_indices as a numpy matrix of zeros and the correct shape (≈ 1 line)
    X_indices = np.zeros((m, max_len))
    
    for i in range(m):                               # loop over training examples
        
        # Convert the ith training sentence in lower case and split is into words. You should get a list of words.
        sentence_words = nlp(X[i].lower())
        
        # Initialize j to 0
        
        if len(sentence_words)> max_len:
            continue
        # Loop over the words of sentence_words
        else:
            j = 0
            for w in sentence_words:

                try:
                    X_indices[i, j] = word_to_index[w.text]
                    j += 1
                except KeyError:
                    continue
                
        print("indexing 已完成:{}/{}".format(i+1,m),end='\r')
                

    return X_indices

In [13]:
w_to_vec_map["cucumber"].shape[0]

50

In [23]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    """
    Creates a Keras Embedding() layer and loads in pre-trained GloVe 50-dimensional vectors.
    
    Arguments:
    word_to_vec_map -- dictionary mapping words to their GloVe vector representation.
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    embedding_layer -- pretrained layer Keras instance
    """
    
    vocab_len = len(word_to_index) + 1                  # adding 1 to fit Keras embedding (requirement)
    emb_dim = word_to_vec_map["cucumber"].shape[0]      # define dimensionality of your GloVe word vectors (= 50)
    
    ### START CODE HERE ###
    # Initialize the embedding matrix as a numpy array of zeros of shape (vocab_len, dimensions of word vectors = emb_dim)
    emb_matrix = np.zeros((vocab_len, emb_dim))
    
    # Set each row "index" of the embedding matrix to be the word vector representation of the "index"th word of the vocabulary
    for word, index in word_to_index.items():
        if word_to_vec_map[word].shape[0]!=50 :
            print("word:", word)
            continue
        emb_matrix[index] = word_to_vec_map[word]

    # Define Keras embedding layer with the correct output/input sizes, make it trainable. Use Embedding(...). Make sure to set trainable=False. 
    embedding_layer = Embedding(vocab_len, emb_dim, trainable = False)
    ### END CODE HERE ###

    # Build the embedding layer, it is required before setting the weights of the embedding layer. Do not modify the "None".
    embedding_layer.build((None,))
    
    # Set the weights of the embedding layer to the embedding matrix. Your layer is now pretrained.
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [24]:
# GRADED FUNCTION: Emojify_V2

def twitter_tech_classify_V1(input_shape, word_to_vec_map, word_to_index):
    """
    Function creating the Emojify-v2 model's graph.
    
    Arguments:
    input_shape -- shape of the input, usually (max_len,)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    model -- a model instance in Keras
    """
    
    # Define sentence_indices as the input of the graph, it should be of shape input_shape and dtype 'int32' (as it contains indices).
    sentence_indices = Input(shape=input_shape, dtype ='int32')
    
    # Create the embedding layer pretrained with GloVe Vectors (≈1 line)
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    
    # Propagate sentence_indices through your embedding layer, you get back the embeddings
    embeddings = embedding_layer(sentence_indices)
    
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a batch of sequences.
    X = LSTM(128, return_sequences = True)(embeddings)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # the returned output should be a single hidden state, not a batch of sequences.
    X = LSTM(128)(X)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X through a Dense layer with softmax activation to get back a batch of 5-dimensional vectors.
    X = Dense(2)(X)
    # Add a softmax activation
    X = Activation(activation='softmax')(X)
    
    # Create Model instance which converts sentence_indices into X.
    model = Model(inputs =sentence_indices ,outputs=X)
    
    
    return model

In [25]:
maxLen = 80

In [26]:

model = twitter_tech_classify_V1((maxLen,), w_to_vec_map, w_to_ix)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 80)                0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 80, 50)            59675700  
_________________________________________________________________
lstm_3 (LSTM)                (None, 80, 128)           91648     
_________________________________________________________________
dropout_3 (Dropout)          (None, 80, 128)           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 258       
__________

In [27]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
X_train_indices = sentences_to_indices(X_train, w_to_ix, maxLen)
X_train_indices.shape[0]

80000

In [81]:
X_train_indices = np.load("x_train_indices.npy")·

In [97]:
len(Y_train_oh)

80000

In [19]:
# np.save("x_train_indices.npy", X_train_indices)

In [91]:
Y_train_oh = convert_to_one_hot(Y_train, C = 2)

In [112]:
model.load_weights("weights.h5")

In [95]:
model.fit(X_train_indices, Y_train_oh, epochs = 10, batch_size = 512, shuffle=True)

Epoch 1/10
18432/80000 [=====>........................] - ETA: 2:21 - loss: 0.7679 - acc: 0.4882

KeyboardInterrupt: 

In [30]:
X_test_indices = sentences_to_indices(X_test, w_to_ix, maxLen)


In [ ]:
Y_test_oh = convert_to_one_hot(Y_test, C = 2)

In [100]:
X_origin_indices = sentences_to_indices(X_test_origin, w_to_ix, maxLen)

In [103]:
Y_origin_oh = convert_to_one_hot(Y_test_origin, C = 2)

798

In [67]:
# np.save("../../data/X_test_indices.npy",X_test_indices)
# np.save("../../data/Y_test_oh.npy",Y_test_oh)

In [145]:
print(model.evaluate(X_test_indices,Y_test_oh, batch_size = 512))

9150/9150 [==============================] - 7s 810us/step
[0.04932591845998999, 0.9899453562986655]


In [146]:
# recall
loss,accuracy = model.evaluate(X_origin_indices[:798],Y_origin_oh[:798], batch_size = 128)

798/798 [==============================] - 1s 973us/step


In [147]:
accuracy

0.9912280701754386

In [61]:
# model.save_weights("weights.h5")

In [87]:
# model.save("model_weights.h5")

In [156]:
s = "There’s an underwater pokéball that helps us study delicate sea creatures without harming them"
X = np.array([s])
X = sentences_to_indices(X, w_to_ix, maxLen)
model.predict(X)

array([[0.9943863 , 0.00561376]], dtype=float32)

In [157]:
list(nlp(s))

[There,
 ’s,
 an,
 underwater,
 pokéball,
 that,
 helps,
 us,
 study,
 delicate,
 sea,
 creatures,
 without,
 harming,
 them]

135